<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [6]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from functions import util
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [7]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
os.environ["HOPSWORKS_API_KEY"] = "bH1DySXH2xPWRdHN.F7jezZJCnUh9rgQGdp4le5lCCAQuSaVWGtXzCNFSfGK6f8ReerVQfxxdJehhO2OU"

project = hopsworks.login()
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1169557
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "china", "city": "shanghai", "street": "songjiang library", "aqicn_url": "https://api.waqi.info/feed/@10214", "latitude": 31.23, "longitude": 121.47, "csv_file": "../../data/songjiang-library-air-quality.csv"}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [8]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [9]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,34.0,china,shanghai,songjiang library,2024-11-18,https://api.waqi.info/feed/@10214


In [10]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 176.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [11]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 31.25°N 121.5°E
Elevation 3.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-18,12.90,0.0,12.096214,36.528946,shanghai
1,2024-11-19,12.90,0.0,6.840000,360.000000,shanghai
2,2024-11-20,13.20,0.0,6.034700,17.354122,shanghai
3,2024-11-21,12.00,0.0,4.680000,337.380096,shanghai
4,2024-11-22,12.10,0.0,9.720000,360.000000,shanghai
5,2024-11-23,11.55,0.0,5.241679,344.054535,shanghai
6,2024-11-24,13.90,0.0,7.895416,65.772263,shanghai
7,2024-11-25,13.75,0.3,11.631956,291.801483,shanghai
8,2024-11-26,6.80,0.0,10.538843,277.853210,shanghai
9,2024-11-27,6.10,0.0,5.495161,301.607452,shanghai


In [12]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 452.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [ ]:
# Insert new data

# air_quality_fg.insert(aq_today_df,overwrite=False, operation="upsert")
# air_quality_df = air_quality_fg.read()##
# air_quality_df['pm25in3day'] = air_quality_df['pm25'].rolling(window=3).mean()##

# Get today's date
today = pd.Timestamp.now(tz='UTC').normalize()
yesterday = (today - pd.Timedelta(days=1)).normalize()
two_days_ago = (today - pd.Timedelta(days=2)).normalize()

air_quality_df = air_quality_fg.read()
yesterday_data = air_quality_df[air_quality_df['date'] == yesterday]
two_days_ago_data = air_quality_df[air_quality_df['date'] == two_days_ago]
avg = two_days_ago_data['pm25'].iloc[0]+yesterday_data['pm25'].iloc[0]+aq_today_df['pm25'].iloc[0]
avg = avg/3.0
aq_today_df['pm25in3day'] = avg   #EXTRA HERE
air_quality_fg.insert(aq_today_df,overwrite=False, operation="upsert")

air_quality_fg

2024-11-18 00:00:00+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.17s) 
                          date  pm25 country      city             street  \
1730 2024-11-18 00:00:00+00:00  34.0   china  shanghai  songjiang library   

                                    url  pm25in3day  
1730  https://api.waqi.info/feed/@10214   61.333333  


In [ ]:
# Insert new data
weather_fg.insert(daily_df)

2024-11-18 10:32:10,046 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1169557/fs/1160260/fg/1352038


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1169557/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f857ec61d00>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 675878
         }
       },
       "result": {
         "observed_value": 4.680000305175781,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-18T09:32:10.000045Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
